In [1]:
import datetime
import re 
import nltk
import pandas as pd 
import numpy as np

In [352]:
ticker_list = ['AAPL','ADBE','AMZN','BA','CMCSA','CSCO','CVX','DIS','FB','GOOGL','INTC','KO','LOW','MCD','MSFT','MU',
               'NFLX','NKE','PEP','SBUX','T','TGT','VZ','WMT','XOM']
def combine(tickerList):                             #will return a combined giant data_frame
    main_data = pd.DataFrame()
    for ticker in ticker_list:
        path = '/Users/Ethan Ohman/Desktop/Equity Research/{0}/{0}.csv'.format(ticker)
        temp_frame = pd.read_csv(path)
        temp_frame = pd.DataFrame({'Bank':temp_frame['Bank'],'Date':temp_frame['Date'],'Position':temp_frame['Position'],'Text':temp_frame['Text'],'Price Target':temp_frame['Price Target']})
        stock = pd.Series([ticker]*len(temp_frame['Bank']))
        temp_frame['Stock'] = stock
        main_data = main_data.append(temp_frame, sort=False)
    return main_data

main = combine(ticker_list)
main.head()

,Bank,Date,Position,Text,Price Target,Stock
0,EVERCORE ISI,21-Nov-2019,Outperform,"Technology | IT Hardware November 21, 2019 Am...",275.0,AAPL
1,MORGAN STANLEY,19-Nov-2019,DNE,"November 19, 2019 04:17 AM GMT MORGAN STANLEY...",DNE,AAPL
2,EVERCORE ISI,17-Nov-2019,Outperform,"Technology | IT Hardware November 17, 2019 Am...",275.0,AAPL
3,EVERCORE ISI,16-Nov-2019,DNE,"Technology | IT Hardware November 16, 2019 Am...",DNE,AAPL
4,JPMORGAN,15-Nov-2019,Overweight,North America Equity Research 15 November 201...,290.0,AAPL


In [132]:
main['Position'].value_counts().head(50)

DNE                       3066
Overweight                 893
BUY                        797
OUTPERFORM                 509
HOLD                       430
Equal-weight               281
Outperform                 209
PERFORM                    175
IN LINE                     90
In Line                     82
UNDERPERFORM                46
Underweight                 41
Neutral                     39
Industry View               29
Bloomberg NYSE: VZ          26
Apple Inc.                  16
LONG                        15
++                          15
Price target $53.00         14
chg        Curr. Y (e       14
Cisco Systems               13
McDonald's Corporation      12
ExxonMobil Corporation       9
NOT RATED                    8
Long                         8
Netflix, Inc.                8
Chevron Corporation          8
OVERWEIGHT                   8
AT&T, Inc.                   8
Facebook, Inc.               8
Amazon.Com, Inc.             8
SELL                         7
In-Line 

In [242]:
bank_list = ['MORGAN STANLEY','EVERCORE ISI','JEFFERIES','JPMORGAN','OPPENHEIMER AND CO']
def inBanks(x):
    for bank in bank_list:
        if(x == bank):
            return True
    return False

def lowerCase(x):
    x = str(x)
    return x.lower()

In [353]:
#create a main database of info
main = main[~(main['Text'].isnull()) & (main['Text'] != 'DNE') & ~(main['Price Target'].isnull())]
main['Price Target'] = main['Price Target'].apply(price_target_clean)
main['Test'] = main['Bank'].apply(inBanks)
main = main[main['Test']]
main.drop('Test',axis=1,inplace=True)
main['Position'] = main['Position'].apply(lowerCase)
print(main['Price Target'].value_counts())


DNE       3108
40.0        86
130.0       79
110.0       75
60.0        74
50.0        72
48.0        67
55.0        65
80.0        61
125.0       54
44.0        49
115.0       48
64.0        48
72.0        47
45.0        46
140.0       43
58.0        42
38.0        42
100.0       41
39.0        41
120.0       39
400.0       38
200.0       38
42.0        38
53.0        36
70.0        35
37.0        35
135.0       35
51.0        34
46.0        34
          ... 
2000.0       1
71.83        1
150.0        1
113.22       1
61.8         1
360.0        1
151.0        1
134.0        1
215.0        1
850.0        1
194.0        1
43.5         1
0.11         1
172.0        1
620.0        1
192.0        1
565.37       1
33.94        1
822.0        1
37.49        1
27.75        1
370.0        1
32.33        1
94.34        1
81.0         1
1970.0       1
547.33       1
509.0        1
31.5         1
670.0        1
Name: Price Target, Length: 441, dtype: int64


In [354]:
main.to_csv('/Users/Ethan Ohman/Desktop/Equity Research/main_data.csv')

In [31]:
#CREATE A DATA FRAME FOR NLP ANALYSIS
main.columns
nlp_df = main.drop(['Bank','Date','Price Target','Stock'],axis=1)
nlp_df = nlp_df[(nlp_df['Position'] != 'dne')]
nlp_df.to_csv('/Users/Ethan Ohman/Desktop/Equity Research/nlp_data.csv')

In [2]:
def date_init(dateString):  #for the date column processing
    date_list = dateString.split('-')
    date_dict = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}
    month = date_dict.get(date_list[1])
    dateString = '{0}/{1}/{2}'.format(date_list[0],month,date_list[2].strip()[2:])
    date = datetime.datetime.strptime(dateString, "%d/%m/%y")
    return date

In [3]:
def position_clean(position):   #change Outpreform to Buy, Neutral and UndertoSell,   Anything else is turned to = null 
    word_dict = {'positive':['overweight','buy','outperform','long'],
                 'negative':['underperform','underweight','sell']}
    for words in word_dict['positive']:
        if(position == words):
            return 'positive'
    for words in word_dict['negative']:
        if(position == words):
            return 'negative'
    return 'dne'
    

In [4]:
def price_target_clean(price):
    if(type(price) == list):
        print('true')
        price = 'DNE'
    try:
        price.index('$')
        price = price.split(' ')
        price = float(price[0][1:])
    except:
        try:
            price.index('NA')
            price = 'DNE'
        except:
            try:
                price.index('(')
                price = price.split(' ')
                price = float(price[0])
            except:
                try:
                    price.index('-')
                    price = price.split('-')
                    price = float(price[0])
                except:
                    try:
                        float(price)
                    except:
                        price = 'DNE'
    return(price)
        

In [5]:
def feature_init(x,y):
    if(x == y):
        return 1
    else:
        return 0

In [6]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
# normalize 
def review_messages(msg):
    # converting messages to lowercase
    msg = msg.lower()
#     msg = [word for word in msg.split() if word not in stopwords]
    return msg

[nltk_data] Downloading package stopwords to C:\Users\Ethan
[nltk_data]     Ohman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
#NLP Model Start here
nlp_data = pd.read_csv('/Users/Ethan Ohman/Desktop/Equity Research/nlp_data.csv')
nlp_data = pd.DataFrame({'Position':nlp_data['Position'], 'Text':nlp_data['Text']})
nlp_data.columns = [col.lower() for col in nlp_data.columns]
nlp_data['position'].value_counts()

positive    2439
negative      94
Name: position, dtype: int64

In [11]:
nlp_data['text'] = nlp_data['text'].apply(review_messages)
nlp_data.head()

,position,text
0,positive,"technology | it hardware november 21, 2019 am..."
1,positive,"technology | it hardware november 17, 2019 am..."
2,positive,north america equity research 15 november 201...
3,positive,north america equity research 06 november 201...
4,positive,"technology | it hardware november 04, 2019 am..."


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(nlp_data['text'], nlp_data['position'], test_size = 0.2, random_state = 1)
# training the vectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)

In [13]:
# train
from sklearn import svm
svm = svm.SVC(C=100)
svm.fit(X_train, y_train)

C:\Users\Ethan Ohman\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [14]:
# test
from sklearn.metrics import confusion_matrix
X_test = vectorizer.transform(X_test)
y_pred = svm.predict(X_test)
print(confusion_matrix(y_test, y_pred))

[[  0  19]
 [  0 488]]


In [15]:
from sklearn.metrics import roc_auc_score
y_score =svm.decision_function(X_test)
print(roc_auc_score(y_test, y_score))

0.9941760138050043


In [16]:
def pred(msg):
    msg = vectorizer.transform([msg])
    prediction = svm.predict(msg)
    return prediction[0]

nlp_data['pred'] = nlp_data['text'].apply(pred)

In [13]:
# nlp_data.head(100)
# nlp_data.to_csv('/Users/Ethan Ohman/Desktop/Equity Research/nlp_test.csv')

In [106]:
# make a method which creates dummy variables for stock, bank, and positivity score
# make a method to return the metric of diff in stock price after a week, two weeks
# just do a linear regression ml model (do multiple ones)

In [17]:
data = pd.read_csv('/Users/Ethan Ohman/Desktop/Equity Research/main_data.csv')
data = data[(data['Price Target'] != 'DNE')]
data['Position'] = data['Text'].apply(pred)   
data['Date'] = data['Date'].apply(date_init)

In [18]:
ticker_list = ['AAPL','ADBE','AMZN','BA','CMCSA','CSCO','CVX','DIS','FB','GOOGL','INTC','KO','LOW','MCD','MSFT','MU',
               'NFLX','NKE','PEP','SBUX','T','TGT','VZ','WMT','XOM']

bank_list = ['MORGAN STANLEY','EVERCORE ISI','JEFFERIES','JPMORGAN','OPPENHEIMER AND CO']

stock_df = pd.DataFrame()
for ticker in ticker_list:
    stock_df[ticker] = data['Stock'].apply(feature_init, args=(ticker,))
stock_df['Date'] = data['Date']
stock_df['Stock'] = data['Stock']
stock_df.shape

(4094, 27)

In [19]:
bank_df = pd.DataFrame()
for bank in data['Bank']:
    bank_df[bank] = data['Bank'].apply(feature_init, args=(bank,))
bank_df['Date'] = data['Date']
bank_df['Stock'] = data['Stock']
bank_df.shape

(4094, 7)

In [20]:
sentiment_df = pd.DataFrame()
for sent in data['Position']:
    sentiment_df[sent] = data['Position'].apply(feature_init, args=(sent,))
sentiment_df['Date'] = data['Date']
sentiment_df['Stock'] = data['Stock']
sentiment_df.head()
sentiment_df.shape

(4094, 3)

In [33]:
def convert_date(date):
    date_l = date.split('/')
    date_s = '{0}/{1}/{2}'.format(date_l[0], date_l[1],date_l[2][2:])
    return datetime.datetime.strptime(date_s, '%d/%m/%y')

In [ ]:
#helper function - get_metric makes sure an initial price is returned

def get_metric(date, price_target, stock_data_frame, section_data_frame, indexer, buffer):
    while(not(abs(stock_data_frame.iloc[indexer]['Close'] - price_target) <= buffer)):
        indexer += -1 #back tracks through the stock price data 
        if(indexer == -1):
            return None 
    # at this point means the data is true
    price_diff = abs(price_target - stock_data_frame.iloc[indexer]['Close'])
    time_diff = abs((stock_data_frame.iloc[indexer]['Date'] - date).days)
    return price_diff*time_diff

In [99]:
#combine our data with yahoo finance to get initial stock prices on a certain day


init_prices = []
temp_l =[]
bank_data = pd.DataFrame()
steck_data = pd.DataFrame()
senti_data = pd.DataFrame()
price_target_data = pd.DataFrame()
position_data = pd.DataFrame()
text_data = pd.DataFrame()

volume = []
metric_list = []
for ticker in ticker_list:
    #init and clean
    stock_data = pd.read_csv('/Users/Ethan Ohman/Desktop/Equity Research/{0}/{0}_price_data.csv'.format(ticker))
    stock_data = stock_data[(stock_data['Date'] != '')]
    stock_data['Date'] = stock_data['Date'].apply(convert_date)
    stock_data = stock_data.iloc[::-1]
    #bound valid data (only deal with a range we know posses proper values)
    min_date = stock_data['Date'].min()
    dummy_frame = data[(data['Stock'] == ticker)&(data['Date'] >= min_date)]
    banks = bank_df[(bank_df['Stock'] == ticker)&(bank_df['Date'] >= min_date)]
    stocks = stock_df[(stock_df['Stock'] == ticker)&(stock_df['Date'] >= min_date)]
    sentiment = sentiment_df[(sentiment_df['Stock'] == ticker)&(sentiment_df['Date'] >= min_date)]
    price_targets = dummy_frame['Price Target']
    position = dummy_frame['Position']
    text = dummy_frame['Text']
    bank_data = bank_data.append(banks, sort=False)
    steck_data= steck_data.append(stocks, sort=False)
    senti_data= senti_data.append(sentiment, sort=False)
    price_target_data = price_target_data.append(price_targets, sort=False)
    position_data = position_data.append(position, sort=False)
    text_data = text_data.append(text,sort=False)
    stock_data_ind = 0
    pt_indexer = 0 
    for date in dummy_frame['Date']:
        while date < stock_data.iloc[stock_data_ind]['Date']:
            stock_data_ind += 1
        if(date == stock_data.iloc[stock_data_ind]['Date']):
            price_target = float(dummy_frame.iloc[pt_indexer]['Price Target'])
            metric = get_metric(date,price_target,stock_data,dummy_frame,stock_data_ind,5)
            metric_list.append(metric)
            volume.append(stock_data.iloc[stock_data_ind]['Volume'])
            temp_l.append(stock_data.iloc[stock_data_ind]['Open'])
            pt_indexer += 1
        else:
            price_target = float(dummy_frame.iloc[pt_indexer]['Price Target'])
            metric = get_metric(date,price_target,stock_data,dummy_frame,stock_data_ind,5)
            metric_list.append(metric)
            volume.append(stock_data.iloc[stock_data_ind+1]['Volume'])
            temp_l.append(stock_data.iloc[stock_data_ind+1]['Open'])
            pt_indexer += 1
    init_prices += temp_l[::-1]
    temp_l.clear()
print(volume)
## VOLUME DF IS CREATED

#     stock_data_ind = 0
#     for date in dummy_frame['Date']:
#         while (date+7) > stock_data.iloc[stock_data_ind]['Date']:
#             stock_data_ind += 1
#         if((date+7) == stock_data.iloc[stock_data_ind]['Date']):
#             week_after


[1609800, 1459700, 2011300, 2762400, 2790200, 2845100, 2845100, 2845100, 2845100, 2320700, 2320700, 2320700, 2281800, 2281800, 2281800, 3511000, 3511000, 3349100, 1372600, 2566300, 3370100, 3992300, 3288100, 2774700, 2774700, 2955000, 2594000, 4097800, 3134500, 2835900, 2835900, 1837000, 2171900, 2776300, 1954500, 2326400, 2196700, 3210200, 3066900, 3066900, 3066900, 2158000, 2158000, 2778400, 2778400, 1378500, 1378500, 1458800, 1826800, 1336400, 1336400, 1644300, 1644300, 1909200, 1909200, 2596800, 5632500, 4529300, 2254200, 1696400, 1313700, 2969300, 2650400, 3903400, 1806000, 2046600, 2056200, 2056200, 5592400, 1867600, 1681100, 2432800, 3150100, 2639300, 2359900, 3992500, 2484800, 1885000, 3705100, 5007200, 2482900, 2482900, 2482900, 2110400, 3822300, 3179500, 3685900, 3663500, 3663500, 3663500, 6110800, 3179400, 3194300, 3971100, 5156700, 3383500, 3498300, 2446700, 4335300, 4335300, 4335300, 4423000, 5626800, 5626800, 4725000, 4725000, 4405100, 5309300, 1735200, 2801400, 4774000, 

In [113]:
p_d = pd.DataFrame()
pr_d = pd.DataFrame()
t_d = pd.DataFrame()
positions = []
text =[]
price = []
for entry in position_data.iloc[0]:
    positions.append(entry)
p_d['Position'] = positions
for entry in price_target_data.iloc[0]:
    price.append(entry)
pr_d['Price Target'] = price
for entry in text_data.iloc[0]:
    text.append(entry)
t_d['Text'] = text


In [461]:
steck_data.shape

(3920, 27)

In [462]:
bank_data.drop(['Date','Stock'], axis=1,inplace = True)
steck_data.drop(['Date','Stock'], axis=1,inplace = True)
senti_data.drop(['Stock'], axis=1,inplace = True) # SENTI HAS DATE

In [467]:
index_list = []
for num in range(3920):
    index_list.append(num)

In [469]:
bank_data.shape
bank_data.index = index_list
bank_data.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            3910, 3911, 3912, 3913, 3914, 3915, 3916, 3917, 3918, 3919],
           dtype='int64', length=3920)

In [479]:

senti_data.index = index_list
senti_data['negative'] = [0]*len(senti_data['positive'])
senti_data.shape

(3920, 3)

In [473]:
steck_data.shape
steck_data.index = index_list
steck_data.head()

,AAPL,ADBE,AMZN,BA,CMCSA,CSCO,CVX,DIS,FB,GOOGL,...,MU,NFLX,NKE,PEP,SBUX,T,TGT,VZ,WMT,XOM
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [376]:
print(init_prices)

[72.209999, 71.709999, 71.790001, 71.540001, 71.540001, 71.540001, 71.540001, 71.540001, 72.779999, 79.139999, 79.629997, 77.760002, 77.760002, 77.760002, 77.25, 77.25, 78.220001, 75.620003, 74.68, 75.08000200000001, 76.839996, 76.839996, 76.339996, 76.339996, 76.339996, 75.610001, 76.400002, 76.43, 74.400002, 79.57, 79.540001, 80.16999799999999, 79.93, 78.900002, 77.879997, 77.879997, 82.16999799999999, 82.650002, 81.379997, 81.379997, 81.379997, 79.900002, 83.389999, 83.389999, 77.599998, 77.599998, 78.639999, 80.790001, 78.629997, 78.629997, 78.610001, 84.91999799999999, 82.199997, 82.410004, 80.449997, 89.0, 88.440002, 88.440002, 88.440002, 89.589996, 91.709999, 95.699997, 89.400002, 88.91999799999999, 87.690002, 87.269997, 87.269997, 87.269997, 85.550003, 88.910004, 98.0, 97.0, 97.0, 96.209999, 95.720001, 95.720001, 95.620003, 99.910004, 96.5, 91.360001, 98.120003, 97.589996, 97.589996, 102.769997, 98.949997, 108.190002, 109.629997, 108.739998, 108.739998, 106.849998, 104.129997, 

In [80]:
#similar to getting initial prices above - just gets the price of that stock x amount of days after date ER report comes out
def get_timeframe(time):
    init_price_pointer = 0
    time_l = []
    holder = []
    interval = datetime.timedelta(float(time))
    for ticker in ticker_list:
        #init vars, lists, and dataframes
        stockfpointer = 0
        pt_pointer = 0
        stock_frame = pd.read_csv('/Users/Ethan Ohman/Desktop/Equity Research/{0}/{0}_price_data.csv'.format(ticker))
        stock_frame['Date'] = stock_frame['Date'].apply(convert_date)
        min_date = stock_frame['Date'].min()
        max_value = stock_frame['Close'].max()
        dummy_frame = data[(data['Stock']== ticker)&(data['Date'] >= min_date)]
        dummy_frame = dummy_frame.iloc[::-1]
        length_frame = len(dummy_frame)
        
        #bound to only valid data
        max_date = dummy_frame['Date'].max() - interval
        dummy_frame = dummy_frame[(dummy_frame['Date'] <= max_date)]
        new_len = len(dummy_frame)
        for date in dummy_frame['Date']:
            price_target = init_prices[init_price_pointer]
            while (date+interval) > stock_frame.iloc[stockfpointer]['Date']:
                stockfpointer += 1
            if((date+interval) == stock_frame.iloc[stockfpointer]['Date']):
                holder.append(float(stock_frame.iloc[stockfpointer]['Close'])-float(price_target))
            else:
                try:
                    holder.append(float(stock_frame.iloc[stockfpointer+1]['Close'])-float(price_target))
                except:
                    holder.append(max_value-price_target)
            pt_pointer += 1
            init_price_pointer += 1
        #imputation
        for rep in range(length_frame-new_len):
            holder.append(max_value-price_target)
#         print(len(holder))
        time_l += holder[::-1]
        holder.clear()
#     print(len(time_l))
    return time_l

In [81]:
time_oneday = get_timeframe(1)
time_threeday = get_timeframe(3)
time_week = get_timeframe(7)
time_twoweek = get_timeframe(14)

In [83]:
print(len(time_oneday))
print(len(time_threeday))
print(len(time_week))
print(len(time_twoweek))

3920
3920
3920
3920


In [120]:
add_on = pd.DataFrame({'Volume':volume, '1 day':time_oneday,'3 day':time_threeday,'1 week':time_week,'2 week':time_twoweek})
add_on.head(100)
f_data = pd.DataFrame()
f_data['Price Target'] = pr_d['Price Target']
for col in senti_data.columns:
    f_data[col] = senti_data[col]
f_data['negative'] = [0]*len(f_data['positive'])
for ticker in ticker_list:
    f_data[ticker] = steck_data[ticker]
for bank in bank_list:
    f_data[bank] = bank_data[bank]
for col in add_on.columns:
    f_data[col] = add_on[col]
f_data['Accuracy Metric'] = metric_list
f_data['Text'] = t_d['Text']

# f_data = pd.concat([senti_data,steck_data,bank_data,add_on], sort=False)
# f_data = f_data.append(senti_data, sort=False,ignore_index=True)
# f_data = f_data.append(steck_data, sort=False,ignore_index=True)
# f_data = f_data.append(bank_data, sort=False,ignore_index=True)
# f_data = f_data.append(add_on, sort=False,ignore_index=True)
f_data.shape
f_data.to_csv('/Users/Ethan Ohman/Desktop/final_data.csv')

In [ ]:
#UNUSED FUNCTION:
# def clean_metric(metric, avg):
#     if(metric == 'MAX'):
#         return avg
#     return metric

# accuracy_metric = []
# temp_series = []
# for ticker in ticker_list: 
#     stock_data = pd.read_csv('/Users/Ethan Ohman/Desktop/Equity Research/{0}/{0}_price_data.csv'.format(ticker))
#     stock_data['Date'] = stock_data['Date'].apply(convert_date)
#     stock_data = stock_data.iloc[::-1]
#     min_date = stock_data['Date'].min()
#     max_stock_price = stock_data['Close'].max()
#     dummy_frame = data[(data['Stock'] == ticker)&(data['Date'] >= min_date)]
    
#     for index in range(len(dummy_frame['Date'])-1,-1,-1):
#         price_target = dummy_frame.iloc[index]['Price Target']
#         dummy_date = dummy_frame.iloc[index]['Date']
#         stock_data_marker = 0
#         if(float(price_target) > float(max_stock_price)): 
#             temp_series.append(None)
#             print('append none1')
#             continue
#         while(not(abs(stock_data.iloc[stock_data_marker]['Close']- float(price_target)) <= 10)): 
#             stock_data_marker += 1
#             if(stock_data_marker == len(dummy_frame['Date'])-1):
#                 temp_series.append(None)
#                 print('append none2')
#                 continue
#         time_diff = (stock_data.iloc[stock_data_marker]['Date']- dummy_date).days
#         print(stock_data.iloc[stock_data_marker]['Date'])
#         print(dummy_date)
#         print(time_diff)
#         price_diff = abs(stock_data.iloc[stock_data_marker]['Close']- float(price_target))
#         if(price_diff == 0):
#             pric_diff = 0.001
#         metric = time_diff*price_diff
#         print(metric)
#         temp_series.append(metric)
#     accuracy_metric += temp_series[::-1]
#     print(accuracy_metric)
#     print(ticker)
# #     print(accuracy_metric)
# metric_series = pd.Series(accuracy_metric)
# metric_series.fillna(metric_series.mean())
    
    
        
        
    
# loop by stock data 
    # make sure to reverse stock data so that it starts high and goes low
    #then loop inside dummy stock data (backwards so it starts from low to high)
    # grab prcice target from dummy stock and main stock data 
    
    #Loop through stock data
    # if Max Price in Time Frame is less than PT, return MAX- as a string, will be filled in later
    # If within 5 dollars of each other, add (timeframe x difference in price target) -- high means not accurate low means accurate
        # if diff is zero - multiply the number by 0.001
    # make data a series and apply formatting 
    # flip list at the end
    # append it to a main series 
    
# MAKE A DATE CATCHER
    
#Fill method
    # if string == max fill with col average